In [1]:
!pip install mlflow optuna plotly

In [2]:
import optuna
import pandas as pd
import mlflow
import plotly
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import mlflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

2024-10-20 13:52:45.020483: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-20 13:52:45.024905: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-20 13:52:45.038808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 13:52:45.063457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 13:52:45.070209: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 13:52:45.088967: I tensorflow/core/platform/cpu_feature_gu

In [5]:
# Load dataset
df_daily = pd.read_csv('../binance_dataset_extract/df_daily.csv')
df_daily

,open_time,open,high,low,close,volume,number_of_trades,volatility,log_returns,SMA_20,SMA_50,RSI
0,2020-12-25,0.000009,0.000168,0.000009,0.000093,31915448.00,226447,NaN,NaN,NaN,NaN,NaN
1,2020-12-26,0.000093,0.000100,0.000060,0.000060,16588695.00,93597,NaN,-0.436802,NaN,NaN,NaN
2,2020-12-27,0.000060,0.000062,0.000039,0.000040,18382334.00,83251,NaN,-0.400495,NaN,NaN,NaN
3,2020-12-28,0.000040,0.000047,0.000040,0.000041,9797253.00,42828,NaN,0.020366,NaN,NaN,NaN
4,2020-12-29,0.000041,0.000041,0.000029,0.000032,11666843.00,48704,NaN,-0.240787,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
687,2022-11-12,0.000033,0.000033,0.000029,0.000030,839696.80,9573,1.363364e-06,-0.080348,0.000030,0.00003,50.834724
688,2022-11-13,0.000030,0.000032,0.000030,0.000032,893477.70,10205,1.291739e-06,0.053754,0.000030,0.00003,58.083838
689,2022-11-14,0.000032,0.000032,0.000029,0.000031,640879.06,4708,1.278455e-06,-0.028610,0.000031,0.00003,54.096049
690,2022-11-15,0.000031,0.000032,0.000031,0.000031,297733.00,2425,1.044792e-06,0.012498,0.000031,0.00003,56.175853


In [6]:
# Define features and target
X = df_daily[['open', 'high', 'low', 'volume', 'number_of_trades']]
y = df_daily['close']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 1. RF Optimization using MLflow and Optuna

In [8]:
def rf_objective(trial):
    # Random Forest Hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
    rf_model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    
    # Cross-validation to make evaluation more robust
    mse = -cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    
    # Log to MLflow
    with mlflow.start_run():
        mlflow.log_param('model', 'RandomForest')
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)
        mlflow.log_param('min_samples_split', min_samples_split)
        mlflow.log_metric('mse', mse)
    
    return mse

In [10]:
# Study and Optimize for Random Forest
rf_study = optuna.create_study(direction='minimize')
rf_study.optimize(rf_objective, n_trials=25)

print(f'Best trial for Random Forest: {rf_study.best_trial.params}')
print(f'Best MSE for Random Forest: {rf_study.best_trial.value}')


[I 2024-10-20 13:56:51,759] A new study created in memory with name: no-name-f8e3e5a5-5cc7-4154-aa23-0964e073626f
[I 2024-10-20 13:56:54,479] Trial 0 finished with value: 1.0255900701532487e-11 and parameters: {'n_estimators': 300, 'max_depth': 5, 'min_samples_split': 8}. Best is trial 0 with value: 1.0255900701532487e-11.
[I 2024-10-20 13:56:55,578] Trial 1 finished with value: 8.798034277116017e-12 and parameters: {'n_estimators': 98, 'max_depth': 7, 'min_samples_split': 2}. Best is trial 1 with value: 8.798034277116017e-12.
[I 2024-10-20 13:56:57,072] Trial 2 finished with value: 8.878245374716752e-12 and parameters: {'n_estimators': 127, 'max_depth': 15, 'min_samples_split': 3}. Best is trial 1 with value: 8.798034277116017e-12.
[I 2024-10-20 13:56:58,540] Trial 3 finished with value: 9.520815106658681e-12 and parameters: {'n_estimators': 146, 'max_depth': 6, 'min_samples_split': 6}. Best is trial 1 with value: 8.798034277116017e-12.
[I 2024-10-20 13:56:59,342] Trial 4 finished wit

Best trial for Random Forest: {'n_estimators': 66, 'max_depth': 6, 'min_samples_split': 2}
Best MSE for Random Forest: 8.478578089017219e-12


## 2. Xgb Optimization using MLflow and Optuna 

In [14]:
def xgb_objective(trial):
    # XGBoost Hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)
    
    xgb_model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        objective='reg:squarederror',
        random_state=42
    )
    
    # Cross-validation for robust evaluation
    mse = -cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

    # Log to MLflow
    with mlflow.start_run():
        mlflow.log_param('model', 'XGBoost')
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)
        mlflow.log_param('learning_rate', learning_rate)
        mlflow.log_param('subsample', subsample)
        mlflow.log_param('colsample_bytree', colsample_bytree)
        mlflow.log_metric('mse', mse)

    return mse

In [16]:
# Study and Optimize for XGBoost
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(xgb_objective, n_trials=25)

print(f'Best trial for XGBoost: {xgb_study.best_trial.params}')
print(f'Best MSE for XGBoost: {xgb_study.best_trial.value}')

[I 2024-10-20 13:58:51,919] A new study created in memory with name: no-name-d94693a7-c835-4740-8353-fa9ed1fda921
/tmp/ipykernel_116/3975406330.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
[I 2024-10-20 13:58:52,231] Trial 0 finished with value: 6.225914682102266e-10 and parameters: {'n_estimators': 221, 'max_depth': 8, 'learning_rate': 0.10784542845363564, 'subsample': 0.6640595878109652, 'colsample_bytree': 0.8205171823743469}. Best is trial 0 with value: 6.225914682102266e-10.
/tmp/ipykernel_116/3975406330.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logun

Best trial for XGBoost: {'n_estimators': 148, 'max_depth': 14, 'learning_rate': 0.0306189109656581, 'subsample': 0.73398107264847, 'colsample_bytree': 0.7000494764179145}
Best MSE for XGBoost: 6.218540333068692e-10


## 3. LSTM Optimization using MLflow and Optuna

In [17]:
# Scale the features for LSTM
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [20]:
import numpy as np

In [21]:
# Create LSTM data with a specific number of timesteps (assuming 10 timesteps)
def create_lstm_data(X, y, timesteps):
    X_lstm, y_lstm = [], []
    for i in range(len(X) - timesteps):
        X_lstm.append(X[i:(i + timesteps), :])
        y_lstm.append(y.iloc[i + timesteps])
    return np.array(X_lstm), np.array(y_lstm)

# Prepare the data with timesteps
timesteps = 10  # You can adjust the number of timesteps based on your needs
X_lstm, y_lstm = create_lstm_data(X_scaled, y, timesteps)

# Train-test split
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

# Reshape the data for LSTM
X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], X_train_lstm.shape[2]))
X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], X_test_lstm.shape[1], X_test_lstm.shape[2]))

In [24]:
from tensorflow.keras.optimizers import Adam

# LSTM Objective function for Optuna optimization
def lstm_objective(trial):
    # LSTM hyperparameters
    units = trial.suggest_int('units', 16, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    epochs = trial.suggest_int('epochs', 10, 50)

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=False, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Early stopping to avoid overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    model.fit(
        X_train_lstm, y_train_lstm,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )

    # Evaluate the model on the test set
    y_pred_lstm = model.predict(X_test_lstm)
    mse = mean_squared_error(y_test_lstm, y_pred_lstm)

    # Log to MLflow
    with mlflow.start_run():
        mlflow.log_param('model', 'LSTM')
        mlflow.log_param('units', units)
        mlflow.log_param('dropout_rate', dropout_rate)
        mlflow.log_param('learning_rate', learning_rate)
        mlflow.log_param('batch_size', batch_size)
        mlflow.log_param('epochs', epochs)
        mlflow.log_metric('mse', mse)

    return mse

In [26]:
# Study and Optimize for LSTM
lstm_study = optuna.create_study(direction='minimize')
lstm_study.optimize(lstm_objective, n_trials=20)

print(f'Best trial for LSTM: {lstm_study.best_trial.params}')
print(f'Best MSE for LSTM: {lstm_study.best_trial.value}')

[I 2024-10-20 14:04:42,231] A new study created in memory with name: no-name-c24b02c7-16df-4e4e-ab01-ceea3d88ac47
/tmp/ipykernel_116/3320099725.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


[I 2024-10-20 14:04:48,363] Trial 0 finished with value: 3.1834030571964327e-06 and parameters: {'units': 101, 'dropout_rate': 0.2699206915918013, 'learning_rate': 0.0017055946346090866, 'batch_size': 20, 'epochs': 49}. Best is trial 0 with value: 3.1834030571964327e-06.
/tmp/ipykernel_116/3320099725.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


[I 2024-10-20 14:04:53,930] Trial 1 finished with value: 8.439575467740129e-07 and parameters: {'units': 79, 'dropout_rate': 0.15691874677299286, 'learning_rate': 0.0024567931695510126, 'batch_size': 52, 'epochs': 36}. Best is trial 1 with value: 8.439575467740129e-07.
/tmp/ipykernel_116/3320099725.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


[I 2024-10-20 14:06:30,688] Trial 18 finished with value: 1.047777562990708e-06 and parameters: {'units': 18, 'dropout_rate': 0.4313966487051731, 'learning_rate': 0.005462693155247491, 'batch_size': 36, 'epochs': 28}. Best is trial 15 with value: 4.956849374239218e-07.
/tmp/ipykernel_116/3320099725.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


[I 2024-10-20 14:06:37,161] Trial 19 finished with value: 4.363882677073568e-07 and parameters: {'units': 105, 'dropout_rate': 0.1612360575332044, 'learning_rate': 0.0026064487029012493, 'batch_size': 28, 'epochs': 20}. Best is trial 19 with value: 4.363882677073568e-07.


Best trial for LSTM: {'units': 105, 'dropout_rate': 0.1612360575332044, 'learning_rate': 0.0026064487029012493, 'batch_size': 28, 'epochs': 20}
Best MSE for LSTM: 4.363882677073568e-07
